In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import os, glob2
import requests, json
from lxml import html
from astropy.time import Time
import pickle
from astropy.table import Table
from astropy.io import ascii
import os
import requests
import json
from datetime import datetime
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy import constants as const
import sys, getopt, argparse
import re
from time import sleep
from astropy.io import fits
from subprocess import call
from lxml import html
import webbrowser as wb
from urllib.error import HTTPError
import xlsxwriter

In [2]:
global TOKEN, BASEURL
GETTOKEN = '8e31426a-f04d-4a32-b470-54102e2d568d'      # Fritz API Key
BASEURL = 'https://fritz.science/'                     # Fritz base url 

In [3]:
API_KEY = "54916f1700966b3bd325fc1189763d86512bda1d"     # TNS API Key

# TNS URLs for real uploads
TNS_BASE_URL = "https://www.wis-tns.org/api/"
upload_url = "https://www.wis-tns.org/api/file-upload"
report_url = "https://www.wis-tns.org/api/bulk-report"
reply_url = "https://www.wis-tns.org/api/bulk-report-reply"

# SANDBOX URLs for TNS upload trials
SAND_TNS_BASE_URL = "https://sandbox.wis-tns.org/api/"
SAND_upload_url = "https://sandbox.wis-tns.org/api/file-upload"
SAND_report_url = "https://sandbox.wis-tns.org/api/bulk-report"
SAND_reply_url = "https://sandbox.wis-tns.org/api/bulk-report-reply"

In [20]:
def api(method, endpoint, data=None):
    ''' Info : Basic API query, takes input the method (eg. GET, POST, etc.), the endpoint (i.e. API url) 
               and additional data for filtering
        Returns : response in json format
        CAUTION! : If the query doesn't go through, try putting the 'data' input in 'data' or 'params' 
                    argument in requests.request call
    '''
    headers = {'Authorization': f'token {GETTOKEN}'}
    response = requests.request(method, endpoint, json=data, headers=headers)
    return response.json()



def get_source_api(ztfname):
    ''' Info : Query a single source, takes input ZTF name
        Returns : all basic data of that source (excludes photometry and spectra, 
                  includes redshift, classification, comments, etc.)
    '''
    url = BASEURL+'api/sources/'+ztfname+'?includeComments=true'
    response = api('GET',url)
    return response['data']    



def get_group_ids(groupnames=['Redshift Completeness Factor', 'Census of the Local Universe Caltech']):
    ''' Info : Query group ids of groups specified
        Input : Name or names of groups in an array []
        Returns : List of group  names and their group ids 
    '''
    
    url = BASEURL+'api/groups'
    headers = {'Authorization': f'token {GETTOKEN}'}
    groupnames = np.atleast_1d(groupnames)
    grpids = []
    for grpname in groupnames:
        response = requests.request('GET',url,params={'name':grpname}, headers=headers).json()
        answer = str(grpname)+' = '+str(response['data'][0]['id'])
        grpids.append(answer)
        
    return grpids



def get_number_of_sources(group_id, date):
    ''' Info : Query number of sources saved in a group after a certain date
        Input : group id, date [yyyy-mm-dd]
        Returns : Number of sources saved after a given date to the specified group
    '''
    
    url = BASEURL+'api/sources?saveSummary=true&group_ids='+group_id+'&savedAfter='+date+'T00:00:00.000001'
    response = api('GET',url)
    return len(response['data']['sources'])



def get_group_sources(group_id, date):
    ''' Info : Query all sources saved in a group after a certain date
        Input : group id, date [yyyy-mm-dd]
        Returns : List of jsons of all sources in group(s)
        Comment : Takes a little time based on the date
    '''
    
    sources = []
    
    for i in range (get_number_of_sources(group_id, date)):
    
        url = BASEURL+'api/sources?saveSummary=true&group_ids='+group_id+'&savedAfter='+date+'T00:00:00.000001'
        response = api('GET',url)
        ztfname = response['data']['sources'][i]['obj_id']
        sources.append(ztfname)
        
    return sources 



def get_total_number_of_sources(group_id):
    ''' Info : Query total number of sources saved in a group
        Input : group id
        Returns : Total number of sources saved in a group
    '''
    
    url = BASEURL+'api/sources?saveSummary=true&group_ids='+group_id
    response = api('GET',url)
    return len(response['data']['sources'])


def get_all_group_sources(group_id):
    ''' Info : Query all sources saved in a group
        Input : group id
        Returns : List of jsons of all sources in group(s)
        Comment : Takes a long time
    '''
    
    sources = []
    
    for i in range (get_number_of_sources(group_id)):
    
        url = BASEURL+'api/sources?saveSummary=true&group_ids='+group_id
        response = api('GET',url)
        ztfname = response['data']['sources'][i]['obj_id']
        sources.append(ztfname)
        
    return sources 


def get_IAUname(ztfname):
    
    ''' Info : Query the TNS name for any source
        Input : ZTFname
        Returns : ATname
    '''
    
    url = BASEURL+'api/alerts_aux/'+ztfname
    response = api('GET',url)
    return response["data"]["cross_matches"]["TNS"]


def get_classification(ztfname):
                        
    ''' Info : Query the classification and classification date for any source
        Input : ZTFname
        Returns : Classification and Classification date
        Comment : You need to choose the classification if there are multiple classifications
    '''
    
    url = BASEURL+'api/sources/'+ztfname+'/classifications'
    response = api('GET',url)
    output = response['data']
    
    if (len(output)< 1):
        classification = "No Classification found"
        classification_date = "None"
    
    if (len(output)==1):
        
        classification = response['data'][0]['classification']
        classification_date = response['data'][0]['created_at'].split('T')[0]
        
    if (len(output) > 1):
        
        classification = []
        classification_date = []
                    
        for i in range (len(output)):
            
            classify = response['data'][i]['classification']  
            classify_date = response['data'][i]['created_at']
                    
            classification.append(classify)
            classification_date.append(classify_date)
                    
        for i in range (len(classification)):
        
            print ((i+1),")", "Classification: ", classification[i],  "\t Classification date:", classification_date[i].split('T')[0])

        user_input = input("Choose classification: ")

        classification = classification[int(user_input)-1]
        classification_date = classification_date[int(user_input)-1].split('T')[0]        
        
    return classification, classification_date


def get_redshift(ztfname):
                    
    ''' Info : Query the redshift for any source
        Input : ZTFname
        Returns : redshift
    '''
    
    url = BASEURL+'api/sources/'+ztfname
    response = api('GET',url)
    
    redshift = response['data']['redshift']
    
    if (redshift == None):
        redshift = "No redshift found"
    
    return redshift


def get_TNS_information(ztfname):
    
    url = BASEURL+'api/sources/'+ztfname
    response = api('GET',url)
        
    IAU = get_IAUname(ztfname)

    if not IAU:
        IAU = "Not reported to TNS"
        
    else:   
        IAU = IAU[0]['name']
              
    clas = get_classification(ztfname)
        
    if clas[1] == 'None':
        clas = "Not classified yet"
        
    else: 
        clas = ('Classification: '+str(clas[0])+','+' Classification date: '+str(clas[1]))
           
    redshift = get_redshift(ztfname)
           
    if redshift == None:
        redshift = "No redshift found"
        
    else:
        redshift = ('redshift:'+str(redshift))        
        
    return ztfname, IAU, clas, redshift

   
def convert_to_jd(date):

    d = Time(date, format='fits')
    dat = d.jd
    return dat


def get_spectrum_api(spectrum_id):
    ''' Info : Query all spectra corresponding to a source, takes input ZTF name
        Returns : list of spectrum jsons
    '''
    url = BASEURL+'api/spectrum/'+str(spectrum_id)
    response = api('GET',url)
    return response


def get_all_spectra_len(ztfname):
    
    url = BASEURL+'api/sources/'+ztfname+'/spectra'
    response = api('GET',url)
    return len(response['data']['spectra'])


def get_all_spectra_id(ztfname):
    ''' Info : Query all spectra corresponding to a source, takes input ZTF name
        Returns : list of spectrum jsons
    '''

    spec_id = []
    
    for i in range (get_all_spectra_len(ztfname)):
    
        url = BASEURL+'api/sources/'+ztfname+'/spectra'
        response = api('GET',url)
        
        specid = response['data']['spectra'][i]['id']
        spec_id.append(specid)
        
    return spec_id


def get_required_spectrum_id(ztfname):
    
    flag = 0
    
    spec = (get_all_spectra_len(ztfname))
    
    name = []
    date = []
    
    if spec == 0:
        
        specid = "No Spectra Found"
        flag = 1
        
    if flag == 0:
        
        spec_id = get_all_spectra_id(ztfname)
        
        for s in range (spec):
            
            url = BASEURL+'api/sources/'+ztfname+'/spectra'
            response = api('GET',url)

            spec_name = response['data']['spectra'][s]['original_file_filename']
            spec_date = response['data']['spectra'][s]['observed_at']
            
            name.append(spec_name)
            date.append(spec_date.split('T')[0])

        print ("Please choose from the following spectra: \n")

        for i in range (len(name)):
            print ((i+1),")", "spectrum name: ", name[i], "spectrum date:", date[i])
            
        wb.open(BASEURL+'source/'+ztfname, new=2) 

        user_input = input("Choose spectrum to upload: ")

        specid = spec_id[int(user_input)-1]

    return specid



def write_ascii_file(ztfname):
    
    specid = get_required_spectrum_id(ztfname)
    
    flag = 0
    
    if (specid == 'No Spectra Found'):
        spectrum_name = 'No Spectra Found'
        print (spectrum_name)
        flag = 1
    
    if flag == 0:
        
        a = get_spectrum_api(specid)

        inst = (a['data']['instrument_name'])
        #print (inst)
    
        if inst == 'SEDM':
        
            header = (a['data']['altdata'])
            path = os.getcwd()

            s = (ztfname+'_'+str(header['OBSDATE'])+'_'+str(inst)+'.ascii')

            with open(path+'/data/'+s,'w') as f:
                f.write(a['data']['original_file_string'])
            f.close()

            #print (s,'\n')
            spectrum_name = s
        
    
        if inst == 'SPRAT':
        
        
            header = (a['data']['altdata'])
            
            path = os.getcwd()

            s = (ztfname+'_'+str(header['OBSDATE'].split('T')[0])+'_'+str(inst)+'.ascii')

            with open(path+'/data/'+s,'w') as f:
                f.write(a['data']['original_file_string'])
            f.close()

            #print (s,'\n')
            spectrum_name = s
            
            
        if inst == 'ALFOSC':
        
            OBSDATE = a['data']['observed_at'].split('T')[0]
            path = os.getcwd()

            s = (ztfname+'_'+str(OBSDATE)+'_'+str(inst)+'.ascii')

            with open(path+'/data/'+s,'w') as f:
                f.write(a['data']['original_file_string'])
            f.close()

            #print (s,'\n')
            spectrum_name = s


        if inst == 'DBSP':
            
            wav = (a['data']['wavelengths'])
            flux = (a['data']['fluxes'])
            err = (a['data']['errors'])
            
            OBSDATE = a['data']['observed_at'].split('T')[0]

            path = os.getcwd()

            s = (ztfname+'_'+str(OBSDATE)+'_'+str(inst)+'.ascii')
            
            if err == None:
                
                with open(path+'/data/'+s,'w') as f:
                
                    for i in range(len(wav)):
                        f.write(str(wav[i])+'\t'+str(flux[i])+'\n')
                f.close()

                #print (s,'\n')
                spectrum_name = s
                
            else:

                with open(path+'/data/'+s,'w') as f:

                    for i in range(len(wav)):
                        f.write(str(wav[i])+'\t'+str(flux[i])+'\t'+str(err[i])+'\n')
                f.close()

                #print (s,'\n')
                spectrum_name = s
            
            
        if inst == 'DIS':

            #obsdate = a['data']['original_file_string'].split('#')[6]
            #a,b = obsdate.split(' ', 1)
            #c,OBSDATE = b.split(' ', 1)
            #OBSDATE = OBSDATE.split('T')[0]
            
            obsdate = a['data']['observed_at']
            OBSDATE = obsdate.split('T')[0]
            
            path = os.getcwd()
            path = path+'/data/'

            s = (ztfname+'_'+str(OBSDATE)+'_'+str(inst)+'.ascii')
            
            a = get_spectrum_api(specid)

            with open(path+s,'w') as f:
                f.write(a['data']['original_file_string'])
            f.close()

            #print (s,'\n')
            spectrum_name = s
            
            
        if inst == 'KAST':

            
            obsdate = a['data']['observed_at']
            OBSDATE = obsdate.split('T')[0]
            
            path = os.getcwd()
            path = path+'/data/'

            s = (ztfname+'_'+str(OBSDATE)+'_'+str(inst)+'.ascii')
            
            a = get_spectrum_api(specid)

            with open(path+s,'w') as f:
                f.write(a['data']['original_file_string'])
            f.close()

            #print (s,'\n')
            spectrum_name = s
            
            
        if inst == 'LRIS':
            
            wav = (a['data']['wavelengths'])
            flux = (a['data']['fluxes'])
            err = (a['data']['errors'])
            
            OBSDATE = a['data']['observed_at'].split('T')[0]

            path = os.getcwd()

            s = (ztfname+'_'+str(OBSDATE)+'_'+str(inst)+'.ascii')
            
            if err == None:
                
                with open(path+'/data/'+s,'w') as f:
                
                    for i in range(len(wav)):
                        f.write(str(wav[i])+'\t'+str(flux[i])+'\n')
                f.close()

                #print (s,'\n')
                spectrum_name = s
                
            else:

                with open(path+'/data/'+s,'w') as f:

                    for i in range(len(wav)):
                        f.write(str(wav[i])+'\t'+str(flux[i])+'\t'+str(err[i])+'\n')
                f.close()

                #print (s,'\n')
                spectrum_name = s
            
    return spectrum_name, specid



def APO(specid):
    
    a = get_spectrum_api(specid)
    inst = (a['data']['instrument_name'])
    
    obsdate = a['data']['original_file_string']['observed_at']
    
    OBSDATE = obsdate
    
    a = get_spectrum_api(specid)

    exptime = a['data']['original_file_string'].split('#')[9]
    a,b = exptime.split(' ', 1)
    c,EXPTIME = b.split(' ', 1)
    
    a = get_spectrum_api(specid)

    observers = (a['data']['original_file_string'].split('#')[10])
    a,b = observers.split(' ', 1)
    c,OBSERVERS = b.split(' ', 1)
    
    a = get_spectrum_api(specid)

    reducers = a['data']['original_file_string'].split('#')[11]
    a,b = reducers.split(' ', 1)
    c,d = b.split(' ', 1)
    REDUCERS,e = d.split('\n', 1)
    
    return OBSDATE.split(' \n')[0], EXPTIME.split(' \n')[0], OBSERVERS.split(' \n')[0], REDUCERS


def pprint(*args, **kwargs):
    """
    slightly more convenient function instead of print(get_pprint)

    params:
        *args (arguments to pass to get_pprint)
        **kwargs (keyword arguments to pass to get_pprint)
    """
    print(get_pprint(*args, **kwargs))
    

def post_comment(ztfname, text):
    
    
    data = {  "obj_id": ztfname,
              "text": text,
           }

    url = BASEURL+'api/comment'
    
    response = api('POST', url, data=data)
    
    return response

def delete_spectrum(spectrum_id):
    url = BASEURL+'api/spectrum/'+spectrum_id
    
    response = api('DELETE', url)
    
    return response


def retrieve_spectrum(spectrum_id):
    url = BASEURL+'api/spectrum/'+spectrum_id
    
    response = api('GET', url)
    
    return response

In [6]:
get_classification('ZTF21aavqwhh')

('Ia', '2021-04-19')

In [40]:
get_required_spectrum_id('ZTF21aaqldsk')

Please choose from the following spectra: 

1 ) spectrum name:  spec_auto_redo131104_lstep1__crr_b_ifu20210408_06_14_37_ZTF21aaqldsk.txt spectrum date: 2021-04-08
2 ) spectrum name:  merge_ZTF21aaqldsk.txt spectrum date: 2021-04-12
3 ) spectrum name:  spec_auto_redo064457_lstep1__crr_b_ifu20210421_05_07_36_ZTF21aaqldsk.txt spectrum date: 2021-04-21
Choose spectrum to upload: 2


2333

In [41]:
get_spectrum_api(2333)['data']

{'instrument_id': 9,
 'modified': '2021-04-21T03:12:17.910062',
 'followup_request_id': None,
 'wavelengths': [3210.4,
  3211.4,
  3212.4,
  3213.4,
  3214.4,
  3215.4,
  3216.4,
  3217.4,
  3218.4,
  3219.4,
  3220.4,
  3221.4,
  3222.4,
  3223.4,
  3224.4,
  3225.4,
  3226.4,
  3227.4,
  3228.4,
  3229.4,
  3230.4,
  3231.4,
  3232.4,
  3233.4,
  3234.4,
  3235.4,
  3236.4,
  3237.4,
  3238.4,
  3239.4,
  3240.4,
  3241.4,
  3242.4,
  3243.4,
  3244.4,
  3245.4,
  3246.4,
  3247.4,
  3248.4,
  3249.4,
  3250.4,
  3251.4,
  3252.4,
  3253.4,
  3254.4,
  3255.4,
  3256.4,
  3257.4,
  3258.4,
  3259.4,
  3260.4,
  3261.4,
  3262.4,
  3263.4,
  3264.4,
  3265.4,
  3266.4,
  3267.4,
  3268.4,
  3269.4,
  3270.4,
  3271.4,
  3272.4,
  3273.4,
  3274.4,
  3275.4,
  3276.4,
  3277.4,
  3278.4,
  3279.4,
  3280.4,
  3281.4,
  3282.4,
  3283.4,
  3284.4,
  3285.4,
  3286.4,
  3287.4,
  3288.4,
  3289.4,
  3290.4,
  3291.4,
  3292.4,
  3293.4,
  3294.4,
  3295.4,
  3296.4,
  3297.4,
  3298.4,
 

In [5]:
get_all_spectra_id('ZTF21aagsxeo')

[1650]

In [5]:
def get_TNS_classification_ID(classification):    

    class_ids = {'Afterglow':23, 'AGN':29, 'CV':27, 'Galaxy':30, 'Gap':60, 'Gap I':61, 'Gap II':62, 'ILRT':25, 'Kilonova':70, 'LBV':24,'M dwarf':210, 'Nova':26, 'Novae':26, 'QSO':31, 'SLSN-I':18, 'Ic-SLSN':18, 'SLSN-II':19, 'SLSN-R':20, 'SN':1, 'I':2, 'Type I':2, 'I-faint':15, 'I-rapid':16, 'Ia':3, 'Ia-norm':3, 'Ia-91bg':103,'Ia-91T':104, 'Ia-CSM':106, 'Ia-pec':100, 'Ia-SC':102, 'Ia-02cx':105,
                'Ib':4, 'Ib-norm':4, 'Ib-Ca-rich':8, 'Ib-pec':107, 'Ib/c':6, 'SN Ibn':9, 'Ibn':9, 'Ic':5, 'Ic-norm':5, 'Ic-BL':7, 'Ic-pec':108, 'II':10, 'Type II':10, 'II-norm':10,
                'II-pec':110, 'IIb':14, 'IIL':12, 'IIn':13, 'IIn-pec':112, 'IIP':11, 'SN impostor':99, 'Std-spec':50, 'TDE':120,
                'Varstar':28, 'WR':200, 'WR-WC':202, 'WR-WN':201, 'WR-WO':203, 'Other':0}
    
    #keys = np.array(class_ids.keys())
    for keys in class_ids:
        if (keys == classification):
            classkey = class_ids[keys]
            return classkey

In [6]:
def get_TNS_instrument_ID(classification):

    inst_ids = {'DBSP':1, 'ALFOSC': 41, 'LRIS': 3, 'DIS': 70, 'SEDM': 149, 'SPRAT': 156, 'GMOS': 6, 'KAST': 10, 'LFC': 2, 'TSPEC': 109}

    #keys = np.array(class_ids.keys())
    for keys in inst_ids:
        if (keys == inst):
            instkey = inst_ids[keys]
            return instkey

In [7]:
class TNSClassificationReport:
    def __init__(self):
        self.name = ''
        self.fitsName = ''
        self.asciiName = ''
        self.classifierName = ''
        self.classificationID = ''
        self.redshift = ''
        self.classificationComments = ''
        self.obsDate = ''
        self.instrumentID = ''
        self.expTime = ''
        self.observers = ''
        self.reducers = ''
        self.specTypeID = ''
        self.spectrumComments = ''
        self.groupID = ''
        self.spec_proprietary_period_value = ''
        self.spec_proprietary_period_units = ''

    def classificationDict(self):
        classificationDict =  {
            'classification_report' : {
                '0' : {
                    'name' : self.name,
                    'classifier' : self.classifierName,
                    'objtypeid' : self.classificationID,
                    'redshift': self.redshift,
                    'groupid' : self.groupID,
                    'remarks' : self.classificationComments,
                    'spectra' : {
                        'spectra-group' : {
                            '0' : {
                                'obsdate' : self.obsDate,
                                'instrumentid' : self.instrumentID,
                                'exptime' : self.expTime,
                                'observer' : self.observers,
                                'reducer' : self.reducers,
                                'spectypeid' : self.specTypeID,
                                'ascii_file' : self.asciiName,
                                'fits_file' : self.fitsName,
                                'remarks' : self.spectrumComments,
                                'spec_proprietary_period' : {
                                    'spec_proprietary_period_value' : self.spec_proprietary_period_value,
                                    'spec_proprietary_period_units' : self.spec_proprietary_period_units
                                }
                            }
                        }
                    }
                }
            }
        }
        return classificationDict

    def classificationJson(self):
        return json.dumps(self.classificationDict())

In [8]:
def upload_to_TNS(filename, base_url = upload_url, api_key = API_KEY, filetype='ascii'):
    
    """
    uploads a file to TNS and returns the response json
    """
    url = base_url
    data = {'api_key' : api_key}

    if filetype is 'ascii':
        files = [('files[]', (filename, open(filename), 'text/plain'))]

    elif filetype is 'fits':
        files = [('files[0]', (filename, open(filename, 'rb'),
                               'application/fits'))]

    if filename:
        response = requests.post(url, data=data, files=files)
        try:
            return response.json()
        except:
            print(url, data, files, response.content, sep='\n')
            return False
    else:
        return {}

In [9]:
def tns_classify(classificationReport, base_url= report_url, api_key=API_KEY):
    """
    submits classification report to TNS and returns the response json
    """
    url = base_url
    data = {'api_key' : api_key, 'data' : classificationReport.classificationJson()}
    response = requests.post(url, data=data).json()
    if not response:
        return False

    res_code = response['id_code']
    report_id = response['data']['report_id']
    print("ID:", report_id)
    print(res_code, response['id_message'], "reporting finished")
    if res_code == 200:
        return report_id
    else:
        print("Result reporting didn't work")
        pprint(response)
        print("re-submit classification, but don't re-upload files")
        return False

In [10]:
def tns_feedback(report_id):
    data = {'api_key': API_KEY, 'report_id': report_id}
    
    response = requests.post(reply_url, data=data).json()
    
    feedback_code = response['id_code']
    print(feedback_code, response['id_message'], "feedback finished")
    if feedback_code == 200:
        return True
    elif feedback_code == 404:
        print("Waiting and retrying...")
        sleep(2)
        try:
            return tns_feedback(report_id)
        except KeyboardInterrupt:
            return False
    elif feedback_code == 400:
        print(response)
        return False
    else:
        # error receiving the feedback from TNS about the upload
        print("Something went wrong with the feedback, but the report may",
              "still have been fine?")
        return False

In [11]:
def get_number(group_id, date):
    ''' Info : Query number of sources saved in a group after a certain date
        Input : group id, date [yyyy-mm-dd]
        Returns : Number of sources saved after a given date to the specified group
    '''
    
    url = BASEURL+'api/sources?saveSummary=true&group_ids='+group_id+'&savedAfter='+date+'T00:00:00.000001'
    response = api('GET',url)
    return len(response['data']['sources'])



def get_sources(group_id, date):
    ''' Info : Query all sources saved in a group after a certain date
        Input : group id, date [yyyy-mm-dd]
        Returns : List of jsons of all sources in group(s)
        Comment : Takes a little time based on the date
    '''
    
    sources = []
    
    for i in range (get_number(group_id, date)):
    
        url = BASEURL+'api/sources?saveSummary=true&group_ids='+group_id+'&savedAfter='+date+'T00:00:00.000001'
        response = api('GET',url)
        ztfname = response['data']['sources'][i]['obj_id']
        sources.append(ztfname)
        
    return sources

def downloadfritzascii(outfile):
    
    date = '2020-11-06' #Specify the date from which you want to check the saved sources
    
    path = 'https://fritz.science/api/sources?group_ids=41&saveSummary=true&savedAfter='+date+'T00:00:00.000001'
 
    response = api('GET',path)
    
    srcs = []
    dates = []
    
    listdir = os.getcwd()
    f = open (listdir+'/'+outfile+'.ascii','w')
    f.write('col1'+'\t'+'col2'+'\n')
    
    for i in range (get_number('41', date)):
        
        source_name = response['data']['sources'][i]['obj_id']
        saved_date = response['data']['sources'][i]['saved_at']
        
        srcs.append(source_name)
        dates.append(saved_date.split('T')[0])
            
    
    #sorted_dates = sorted(dates, reverse = True)
    #Sources = [x for _,x in sorted(zip(dates,srcs), reverse=True)]
    
    output = sorted(zip(dates, srcs), reverse=True)
    
    for i in range (get_number('41', date)):
    
        f.write(output[i][1]+'\t'+output[i][0]+'\n')
        
    f.close()
        
    return

In [21]:
#downloadfritzascii('RCF_sources') #download the updated list of sources saved to RCF in descending order

#f = ascii.read("RCF_sources_copy.ascii") #ascii file containing the names of sources and their saved dates
#sources = f['col1']

sources = ['ZTF21aaxhzru', 'ZTF21aawzsew', 'ZTF21aavuprl', 'ZTF21aartgiv']
           
           
           
for source in sources:
    
    flag = 0
    ztfname = source
    
    for i in range (len(get_source_api(source)['comments'])):
        
        comment = get_source_api(source)['comments'][i]['text']
        
        if comment == 'Uploaded to TNS':
            flag = 1
        
    if flag == 0:
        
        info = get_TNS_information(ztfname)
        
        if info[2] == 'Not classified yet':         #Check if classified
            flag =1
            continue  


        a,b = (info[2]).split(',', 1)               #This is just to get the classification date
        c,d = b.split(':', 1)
        e,class_date = d.split(' ', 1)


        k,l = (info[2]).split(':', 1)               #This is just to get the classification
        m,n = l.split(',', 1)
        o,classify = m.split(' ', 1)

        print (classify, class_date)


        print (info, '\n')
        print ("Do you want to proceed with the report?")

        user_input = input("y/n: ")

        if user_input == 'y':

            spectrum_info = write_ascii_file(ztfname) #returns "spectrum_name"

            spectrum_name = spectrum_info[0]

            if spectrum_name == 'No Spectra Found':
                flag = 1
                continue

            if flag == 0:

                path = os.getcwd()

                specfile = (path+'/data/'+spectrum_name)

                files = specfile

                specid = spectrum_info[1]

                a = get_spectrum_api(specid)

                inst = (a['data']['instrument_name'])


                if inst == 'SEDM':

                    classifiers = 'A. Dahiwale, C. Fremling(Caltech) on behalf of the Zwicky Transient Facility (ZTF)'### Change accordingly
                    source_group = 48 ### Require source group id from drop down list, 0 is for None
                    spectypes = np.array(['object','host','sky','arcs','synthetic'])

                    #proprietary_period = int(input("Proprietary period in years:", x)
                    proprietary_period = '0'
                    proprietary_units = "years"
                    spec_comments =''
                    classification_comments = ''
                    spectype='object'
                    spectype_id = ['object', 'host', 'sky', 'arcs', 'synthetic'].index(spectype) + 1

                    header = (a['data']['altdata'])
                    obsdate = str((header['UTC']).split('T')[0])+' '+str((header['UTC']).split('T')[1])

                    classificationReport = TNSClassificationReport()
                    classificationReport.name = get_IAUname(ztfname)[0]['name'][3:]  
                    classificationReport.fitsName = ''
                    classificationReport.asciiName = spectrum_name
                    classificationReport.classifierName = classifiers
                    classificationReport.classificationID = get_TNS_classification_ID(classify)
                    classificationReport.redshift = get_redshift(ztfname)
                    classificationReport.classificationComments = classification_comments
                    classificationReport.obsDate = obsdate
                    classificationReport.instrumentID = get_TNS_instrument_ID(inst)
                    classificationReport.expTime = (header['EXPTIME'])
                    classificationReport.observers = 'SEDmRobot'
                    classificationReport.reducers = (header['REDUCER'])
                    classificationReport.specTypeID = spectype_id
                    classificationReport.spectrumComments = spec_comments
                    classificationReport.groupID = source_group
                    classificationReport.spec_proprietary_period_value = proprietary_period
                    classificationReport.spec_proprietary_period_units = proprietary_units
                    print(classificationReport.classificationDict())

                    #pprint(classificationReport.fill(), tab='  ')
                    proceed = input("\nProceed with classification and upload? ([y]/n) : ")
                    if proceed == 'y' and not proceed.strip() == '':

                        # ASCII FILE UPLOAD
                        print ("\n")
                        response = upload_to_TNS(files)
                        print (response)

                        if not response:
                            print("File upload didn't work")
                            print(response)
                            #return False

                        print(response['id_code'], response['id_message'],
                              "\nSuccessfully uploaded ascii spectrum")
                        #classificationReport.asciiName = response['data'][-1]

                        report_id = tns_classify(classificationReport)
                        post_comment(ztfname, 'Uploaded to TNS')
                        tns_feedback(report_id)


                if inst == 'SPRAT':

                    classifiers = 'D. A. Perley, K. Taggart (LJMU), A. Dahiwale, C. Fremling (Caltech) on behalf of the Zwicky Transient Facility (ZTF)'### Change accordingly
                    source_group = 48 ### Require source group id from drop down list, 0 is for None
                    spectypes = np.array(['object','host','sky','arcs','synthetic'])

                    #proprietary_period = int(input("Proprietary period in years:", x)
                    proprietary_period = '0'
                    proprietary_units = "years"
                    spec_comments =''
                    classification_comments = ''
                    spectype='object'
                    spectype_id = ['object', 'host', 'sky', 'arcs', 'synthetic'].index(spectype) + 1

                    header = (a['data']['altdata'])
                    obsdate = str(header['OBSDATE'].split('T')[0])+' '+str(header['OBSDATE'].split('T')[1])

                    classificationReport = TNSClassificationReport()
                    classificationReport.name = get_IAUname(ztfname)[0]['name'][3:] 
                    classificationReport.fitsName = ''
                    classificationReport.asciiName = spectrum_name
                    classificationReport.classifierName = classifiers
                    classificationReport.classificationID = get_TNS_classification_ID(classify)
                    classificationReport.redshift = get_redshift(ztfname)
                    classificationReport.classificationComments = classification_comments
                    classificationReport.obsDate = obsdate
                    classificationReport.instrumentID = get_TNS_instrument_ID(inst)
                    classificationReport.expTime = (header['EXPTIME'])
                    classificationReport.observers = 'LTRobot'
                    classificationReport.reducers = 'D. Perley'
                    classificationReport.specTypeID = spectype_id
                    classificationReport.spectrumComments = spec_comments
                    classificationReport.groupID = source_group
                    classificationReport.spec_proprietary_period_value = proprietary_period
                    classificationReport.spec_proprietary_period_units = proprietary_units
                    print(classificationReport.classificationDict())

                    #pprint(classificationReport.fill(), tab='  ')
                    proceed = input("\nProceed with classification and upload? ([y]/n) : ")
                    if proceed == 'y' and not proceed.strip() == '':

                        # ASCII FILE UPLOAD
                        print ("\n")
                        response = upload_to_TNS(files)
                        print (response)

                        if not response:
                            print("File upload didn't work")
                            print(response)
                            #return False

                        print(response['id_code'], response['id_message'],
                              "\nSuccessfully uploaded ascii spectrum")
                        #classificationReport.asciiName = response['data'][-1]

                        report_id = tns_classify(classificationReport)
                        post_comment(ztfname, 'Uploaded to TNS')
                        tns_feedback(report_id)



                if inst == 'ALFOSC':

                    classifiers = 'A. Dahiwale, C. Fremling(Caltech) on behalf of the Zwicky Transient Facility (ZTF)'### Change accordingly
                    source_group = 48 ### Require source group id from drop down list, 0 is for None
                    spectypes = np.array(['object','host','sky','arcs','synthetic'])

                    #proprietary_period = int(input("Proprietary period in years:", x)
                    proprietary_period = '0'
                    proprietary_units = "years"
                    spec_comments =''
                    classification_comments = ''
                    spectype='object'
                    spectype_id = ['object', 'host', 'sky', 'arcs', 'synthetic'].index(spectype) + 1

                    header = (a['data']['altdata'])
                    obsdate = str(a['data']['observed_at'].split('T')[0])+' '+str(a['data']['observed_at'].split('T')[1])

                    classificationReport = TNSClassificationReport()
                    classificationReport.name = get_IAUname(ztfname)[0]['name'][3:]  
                    classificationReport.fitsName = ''
                    classificationReport.asciiName = spectrum_name
                    classificationReport.classifierName = classifiers
                    classificationReport.classificationID = get_TNS_classification_ID(classify)
                    classificationReport.redshift = get_redshift(ztfname)
                    classificationReport.classificationComments = classification_comments
                    classificationReport.obsDate = obsdate
                    classificationReport.instrumentID = get_TNS_instrument_ID(inst)
                    classificationReport.expTime = (header['EXPTIME'])
                    classificationReport.observers = (str(a['data']['observers'][0]['first_name'])+' '+str(a['data']['observers'][0]['last_name']))
                    classificationReport.reducers = (str(a['data']['reducers'][0]['first_name'])+' '+str(a['data']['reducers'][0]['last_name']))
                    classificationReport.specTypeID = spectype_id
                    classificationReport.spectrumComments = spec_comments
                    classificationReport.groupID = source_group
                    classificationReport.spec_proprietary_period_value = proprietary_period
                    classificationReport.spec_proprietary_period_units = proprietary_units
                    print(classificationReport.classificationDict())

                    #pprint(classificationReport.fill(), tab='  ')
                    proceed = input("\nProceed with classification and upload? ([y]/n) : ")
                    if proceed == 'y' and not proceed.strip() == '':

                        # ASCII FILE UPLOAD
                        print ("\n")
                        response = upload_to_TNS(files)
                        print (response)

                        if not response:
                            print("File upload didn't work")
                            print(response)
                            #return False

                        print(response['id_code'], response['id_message'],
                              "\nSuccessfully uploaded ascii spectrum")
                        #classificationReport.asciiName = response['data'][-1]

                        report_id = tns_classify(classificationReport)
                        post_comment(ztfname, 'Uploaded to TNS')
                        tns_feedback(report_id)



                if inst == 'DBSP':

                    classifiers = 'A. Dahiwale, C. Fremling(Caltech) on behalf of the Zwicky Transient Facility (ZTF)'### Change accordingly
                    source_group = 48 ### Require source group id from drop down list, 0 is for None
                    spectypes = np.array(['object','host','sky','arcs','synthetic'])

                    #proprietary_period = int(input("Proprietary period in years:", x)
                    proprietary_period = '0'
                    proprietary_units = "years"
                    spec_comments =''
                    classification_comments = ''
                    spectype='object'
                    spectype_id = ['object', 'host', 'sky', 'arcs', 'synthetic'].index(spectype) + 1

                    OBSDATE = str(a['data']['observed_at'].split('T')[0])+' '+str(a['data']['observed_at'].split('T')[1])

                    classificationReport = TNSClassificationReport()
                    classificationReport.name = get_IAUname(ztfname)[0]['name'][3:]  
                    classificationReport.fitsName = ''
                    classificationReport.asciiName = spectrum_name
                    classificationReport.classifierName = classifiers
                    classificationReport.classificationID = get_TNS_classification_ID(classify)
                    classificationReport.redshift = get_redshift(ztfname)
                    classificationReport.classificationComments = classification_comments
                    classificationReport.obsDate = OBSDATE
                    classificationReport.instrumentID = get_TNS_instrument_ID(inst)
                    classificationReport.expTime = '450'
                    classificationReport.observers = (str(a['data']['observers'][0]['first_name'])+' '+str(a['data']['observers'][0]['last_name']))
                    classificationReport.reducers = (str(a['data']['reducers'][0]['first_name'])+' '+str(a['data']['reducers'][0]['last_name']))
                    classificationReport.specTypeID = spectype_id
                    classificationReport.spectrumComments = spec_comments
                    classificationReport.groupID = source_group
                    classificationReport.spec_proprietary_period_value = proprietary_period
                    classificationReport.spec_proprietary_period_units = proprietary_units
                    print(classificationReport.classificationDict())

                    #pprint(classificationReport.fill(), tab='  ')
                    proceed = input("\nProceed with classification and upload? ([y]/n) : ")
                    if proceed == 'y' and not proceed.strip() == '':

                        # ASCII FILE UPLOAD
                        print ("\n")
                        response = upload_to_TNS(files)
                        print (response)

                        if not response:
                            print("File upload didn't work")
                            print(response)
                            #return False

                        print(response['id_code'], response['id_message'],
                              "\nSuccessfully uploaded ascii spectrum")
                        #classificationReport.asciiName = response['data'][-1]

                        report_id = tns_classify(classificationReport)
                        post_comment(ztfname, 'Uploaded to TNS')
                        tns_feedback(report_id)  


                if inst == 'LRIS':

                    classifiers = 'A. Dahiwale, C. Fremling(Caltech) on behalf of the Zwicky Transient Facility (ZTF)'### Change accordingly
                    source_group = 48 ### Require source group id from drop down list, 0 is for None
                    spectypes = np.array(['object','host','sky','arcs','synthetic'])

                    header = (a['data']['altdata'])

                    #proprietary_period = int(input("Proprietary period in years:", x)
                    proprietary_period = '0'
                    proprietary_units = "years"
                    spec_comments =''
                    classification_comments = ''
                    spectype='object'
                    spectype_id = ['object', 'host', 'sky', 'arcs', 'synthetic'].index(spectype) + 1

                    OBSDATE = str(a['data']['observed_at'].split('T')[0])+' '+str(a['data']['observed_at'].split('T')[1])

                    classificationReport = TNSClassificationReport()
                    classificationReport.name = get_IAUname(ztfname)[0]['name'][3:]  
                    classificationReport.fitsName = ''
                    classificationReport.asciiName = spectrum_name
                    classificationReport.classifierName = classifiers
                    classificationReport.classificationID = get_TNS_classification_ID(classify)
                    classificationReport.redshift = get_redshift(ztfname)
                    classificationReport.classificationComments = classification_comments
                    classificationReport.obsDate = OBSDATE
                    classificationReport.instrumentID = get_TNS_instrument_ID(inst)
                    #classificationReport.expTime = '300'
                    classificationReport.observers = (str(a['data']['observers'][0]['first_name'])+' '+str(a['data']['observers'][0]['last_name']))
                    classificationReport.reducers = (str(a['data']['reducers'][0]['first_name'])+' '+str(a['data']['reducers'][0]['last_name']))
                    classificationReport.specTypeID = spectype_id
                    classificationReport.spectrumComments = spec_comments
                    classificationReport.groupID = source_group
                    classificationReport.spec_proprietary_period_value = proprietary_period
                    classificationReport.spec_proprietary_period_units = proprietary_units
                    print(classificationReport.classificationDict())

                    #pprint(classificationReport.fill(), tab='  ')
                    proceed = input("\nProceed with classification and upload? ([y]/n) : ")
                    if proceed == 'y' and not proceed.strip() == '':

                        # ASCII FILE UPLOAD
                        print ("\n")
                        response = upload_to_TNS(files)
                        print (response)

                        if not response:
                            print("File upload didn't work")
                            print(response)
                            #return False

                        print(response['id_code'], response['id_message'],
                              "\nSuccessfully uploaded ascii spectrum")
                        #classificationReport.asciiName = response['data'][-1]

                        report_id = tns_classify(classificationReport)
                        post_comment(ztfname, 'Uploaded to TNS')
                        tns_feedback(report_id)  


                if inst == 'DIS':

                    classifiers = 'Melissa L. Graham (UW), A. Dahiwale, C. Fremling(Caltech) on behalf of the Zwicky Transient Facility (ZTF)'### Change accordingly
                    source_group = 48 ### Require source group id from drop down list, 0 is for None
                    spectypes = np.array(['object','host','sky','arcs','synthetic'])
                    #proprietary_period = int(input("Proprietary period in years:", x)
                    proprietary_period = '0'
                    proprietary_units = "years"
                    spec_comments =''
                    classification_comments = ''
                    spectype='object'
                    spectype_id = ['object', 'host', 'sky', 'arcs', 'synthetic'].index(spectype) + 1

                    #obsdate = APO(specid)[0]  
                    #exptime = APO(specid)[1]
                    #observers = APO(specid)[2]
                    #reducers = APO(specid)[3]

                    obsdate = str(a['data']['observed_at'].split('T')[0])+' '+str(a['data']['observed_at'].split('T')[1])

                    classificationReport = TNSClassificationReport()
                    classificationReport.name = get_IAUname(ztfname)[0]['name'][3:]  
                    classificationReport.fitsName = ''
                    classificationReport.asciiName = spectrum_name
                    classificationReport.classifierName = classifiers
                    classificationReport.classificationID = get_TNS_classification_ID(classify)
                    classificationReport.redshift = get_redshift(ztfname)
                    classificationReport.classificationComments = classification_comments
                    classificationReport.obsDate = obsdate
                    classificationReport.instrumentID = get_TNS_instrument_ID(inst)
                    #classificationReport.expTime = exptime
                    #classificationReport.observers = observers
                    #classificationReport.reducers = reducers
                    classificationReport.observers = (str(a['data']['observers'][0]['first_name'])+' '+str(a['data']['observers'][0]['last_name']))
                    classificationReport.reducers = (str(a['data']['reducers'][0]['first_name'])+' '+str(a['data']['reducers'][0]['last_name']))
                    classificationReport.specTypeID = spectype_id
                    classificationReport.spectrumComments = spec_comments
                    classificationReport.groupID = source_group
                    classificationReport.spec_proprietary_period_value = proprietary_period
                    classificationReport.spec_proprietary_period_units = proprietary_units
                    print(classificationReport.classificationDict())

                    #pprint(classificationReport.fill(), tab='  ')
                    proceed = input("\nProceed with classification and upload? ([y]/n) : ")
                    if proceed == 'y' and not proceed.strip() == '':

                        #ASCII FILE UPLOAD
                        print ("\n")
                        response = upload_to_TNS(files)
                        print (response)

                        if not response:
                            print("File upload didn't work")
                            print(response)
                            #return False

                        print(response['id_code'], response['id_message'],
                              "\nSuccessfully uploaded ascii spectrum")
                        #classificationReport.asciiName = response['data'][-1]

                        report_id = tns_classify(classificationReport)
                        tns_feedback(report_id)
                        post_comment(ztfname, 'Uploaded to TNS')
                        
                        
                if inst == 'KAST':

                    #classifiers = 'Thomas G. Brink, Kishore C. Patra, Thomas de Jaeger, WeiKang Zheng, Benjamin E. Stahl, Alexei V. Filippenko (UC Berkeley), A. Dahiwale, C. Fremling (Caltech) on behalf of the Zwicky Transient Facility (ZTF)'### Change accordingly
                    #classifiers = 'A. Dahiwale, C. Fremling(Caltech) on behalf of the Zwicky Transient Facility (ZTF)'### Change accordingly
                    source_group = 48 ### Require source group id from drop down list, 0 is for None
                    spectypes = np.array(['object','host','sky','arcs','synthetic'])
                    #proprietary_period = int(input("Proprietary period in years:", x)
                    proprietary_period = '0'
                    proprietary_units = "years"
                    spec_comments =''
                    classification_comments = ''
                    spectype='object'
                    spectype_id = ['object', 'host', 'sky', 'arcs', 'synthetic'].index(spectype) + 1


                    obsdate = str(a['data']['observed_at'].split('T')[0])+' '+str(a['data']['observed_at'].split('T')[1])

                    classificationReport = TNSClassificationReport()
                    classificationReport.name = get_IAUname(ztfname)[0]['name'][3:]  
                    classificationReport.fitsName = ''
                    classificationReport.asciiName = spectrum_name
                    classificationReport.classifierName = classifiers
                    classificationReport.classificationID = get_TNS_classification_ID(classify)
                    classificationReport.redshift = get_redshift(ztfname)
                    classificationReport.classificationComments = classification_comments
                    classificationReport.obsDate = obsdate
                    classificationReport.instrumentID = get_TNS_instrument_ID(inst)
                    #classificationReport.expTime = exptime
                    classificationReport.observers = 'Mike Rich'
                    classificationReport.reducers = 'Mike Rich'
                    #classificationReport.observers = (str(a['data']['observers'][0]['first_name'])+' '+str(a['data']['observers'][0]['last_name']))
                    #classificationReport.reducers = (str(a['data']['reducers'][0]['first_name'])+' '+str(a['data']['reducers'][0]['last_name']))
                    classificationReport.specTypeID = spectype_id
                    classificationReport.spectrumComments = spec_comments
                    classificationReport.groupID = source_group
                    classificationReport.spec_proprietary_period_value = proprietary_period
                    classificationReport.spec_proprietary_period_units = proprietary_units
                    print(classificationReport.classificationDict())

                    #pprint(classificationReport.fill(), tab='  ')
                    proceed = input("\nProceed with classification and upload? ([y]/n) : ")
                    if proceed == 'y' and not proceed.strip() == '':

                        #ASCII FILE UPLOAD
                        print ("\n")
                        response = upload_to_TNS(files)
                        print (response)

                        if not response:
                            print("File upload didn't work")
                            print(response)
                            #return False

                        print(response['id_code'], response['id_message'],
                              "\nSuccessfully uploaded ascii spectrum")
                        #classificationReport.asciiName = response['data'][-1]

                        report_id = tns_classify(classificationReport)
                        tns_feedback(report_id)
                        post_comment(ztfname, 'Uploaded to TNS')
                        
                        #print('deleting files now')
                        #call(['rm', all_info['asciifile']])

Ic-BL 2021-04-30
('ZTF21aartgiv', 'AT 2021hyz', 'Classification: Ic-BL, Classification date: 2021-04-30', 'redshift:0.0461') 

Do you want to proceed with the report?
y/n: y
Please choose from the following spectra: 

1 ) spectrum name:  None spectrum date: 2020-04-16
2 ) spectrum name:  spec_auto_contsep_lstep1__crr_b_ifu20210405_08_44_42_ZTF21aartgiv.txt spectrum date: 2021-04-05
3 ) spectrum name:  spec_auto_robot_lstep1__crr_b_ifu20210405_08_44_42_ZTF21aartgiv.txt spectrum date: 2021-04-05
4 ) spectrum name:  spec_auto_robot_lstep1__crr_b_ifu20210407_08_41_16_ZTF21aartgiv.txt spectrum date: 2021-04-07
5 ) spectrum name:  spec_auto_robot_lstep1__crr_b_ifu20210415_04_17_49_ZTF21aartgiv.txt spectrum date: 2021-04-15
6 ) spectrum name:  spec_auto_robot_lstep1__crr_b_ifu20210430_03_59_03_ZTF21aartgiv.txt spectrum date: 2021-04-30
Choose spectrum to upload: 6
{'classification_report': {'0': {'name': '2021hyz', 'classifier': 'A. Dahiwale, C. Fremling(Caltech) on behalf of the Zwicky Trans

In [17]:
def get_new_classification(ztfname):
                        
    ''' Info : Query the classification and classification date for any source
        Input : ZTFname
        Returns : Classification and Classification date
        Comment : You need to choose the classification if there are multiple classifications
    '''
    try:
        
        url = BASEURL+'api/sources/'+ztfname+'/classifications'
        response = api('GET',url)
        output = response['data']

        if (len(output)< 1):
            classification = "Not Classified"
            classification_date = "None"

        if (len(output) >= 1):

            classification = response['data'][(len(output)-1)]['classification']  
            classification_date = response['data'][(len(output)-1)]['created_at']
            
    except KeyError as e:
        classification = "Error"
        classification_date = "Error"
        
    return classification, classification_date

def get_new_IAUname(ztfname):
    
    ''' Info : Query the TNS name for any source
        Input : ZTFname
        Returns : ATname
    '''
    try:
        
        url = BASEURL+'api/alerts_aux/'+ztfname
        response = api('GET',url)
        IAU = response['data']['cross_matches']['TNS']
 
        
        if not IAU:
            IAU = "Not reported to TNS"

        else:   
            IAU = IAU[0]['name']

    except KeyError as e:
        IAU = "Error"
        
    return IAU


def get_number(group_id, date):
    ''' Info : Query number of sources saved in a group after a certain date
        Input : group id, date [yyyy-mm-dd]
        Returns : Number of sources saved after a given date to the specified group
    '''
    
    url = BASEURL+'api/sources?saveSummary=true&group_ids='+group_id+'&savedAfter='+date+'T00:00:00.000001'
    response = api('GET',url)
    return len(response['data']['sources'])



def get_sources(group_id, date):
    ''' Info : Query all sources saved in a group after a certain date
        Input : group id, date [yyyy-mm-dd]
        Returns : List of jsons of all sources in group(s)
        Comment : Takes a little time based on the date
    '''
    
    sources = []
    
    for i in range (get_number(group_id, date)):
    
        url = BASEURL+'api/sources?saveSummary=true&group_ids='+group_id+'&savedAfter='+date+'T00:00:00.000001'
        response = api('GET',url)
        ztfname = response['data']['sources'][i]['obj_id']
        sources.append(ztfname)
        
    return sources

def get_comment(ztfname):
    
    flag = 0
    
    for i in range (len(get_source_api(ztfname)['comments'])):
    
        TNS_comment = get_source_api(ztfname)['comments'][i]['text']

        if TNS_comment == 'Uploaded to TNS':
            comment = 'Yes'
            flag = 1
            
            
    if flag == 0:
        comment = 'No'
            
    return comment

In [18]:
def sourceclassification(outfile):
    
    dat = '2020-11-06' #Specify the date from which you want to check the saved sources
    #dat = '2021-01-10'
    path = 'https://fritz.science/api/sources?group_ids=41&saveSummary=true&savedAfter='+dat+'T00:00:00.000001'
 
    response = api('GET',path)
    
    srcs = []
    dates = []
    classify = []
    class_date = []
    TNS = []
    
    #annotation = []
    
    listdir = os.getcwd()
    f = open (listdir+'/'+outfile+'.ascii','w')
    f.write('Source Name'+'\t'+'TNS Name'+'\t'+'Saved Date'+'\t'+'Classification'+'\t'+'Classification Date'+'\n')
    
    for i in range (get_number('41', dat)):
            
        source_name = response['data']['sources'][i]['obj_id']
        saved_date = response['data']['sources'][i]['saved_at']
        classification = get_new_classification(source_name)[0]
        date = get_new_classification(source_name)[1]
        IAU = get_new_IAUname(source_name)
        
        #fritz_comment = get_comment(source_name)
            
        print (i, source_name)

        srcs.append(source_name)
        TNS.append(IAU)
        dates.append(saved_date.split('T')[0])
        classify.append(classification)
        class_date.append(date.split('T')[0])
        
        #annotation.append(fritz_comment)

        
    output = sorted(zip(class_date, srcs, TNS, dates, classify), reverse=True)
    #output = sorted(zip(class_date, srcs, dates, classify), reverse=True)

    for i in range (get_number('41', dat)):

        f.write(output[i][1]+'\t'+output[i][2]+'\t'+output[i][3]+'\t'+output[i][4]+'\t'+output[i][0]+'\n')
        #f.write(output[i][1]+'\t'+output[i][2]+'\t'+output[i][3]+'\t'+output[i][0]+'\n')

    f.close()
        
    return

In [19]:
sourceclassification('classification_0430')

0 ZTF18abtoued
1 ZTF20acmzfsz
2 ZTF20acoxmsr
3 ZTF20acoanlv
4 ZTF20acivqxo
5 ZTF20acmgrda
6 ZTF20acpbboa
7 ZTF18aahjbii
8 ZTF20acpevli
9 ZTF20acpgnsn
10 ZTF20acpgnej
11 ZTF18adhzhfw
12 ZTF20acozxxt
13 ZTF19aanbpus
14 ZTF18abrfccm
15 ZTF20acpdrab
16 ZTF19aczeomw
17 ZTF20acggkic
18 ZTF19acvieqw
19 ZTF20aclkdlu
20 ZTF20aboquuv
21 ZTF20acgcice
22 ZTF20abupbub
23 ZTF20abvyare
24 ZTF20abzcfqn
25 ZTF20acjynpx
26 ZTF20abzpxxu
27 ZTF20acegtsw
28 ZTF20acinuir
29 ZTF20abyjoxb
30 ZTF20abwifwx
31 ZTF20aclmssm
32 ZTF20acnvprg
33 ZTF20acouofb
34 ZTF20abzeynv
35 ZTF20achdusl
36 ZTF20acgyvgs
37 ZTF20acpjlon
38 ZTF20acopmey
39 ZTF20acjryhf
40 ZTF20acfmxbj
41 ZTF20acounqo
42 ZTF20acmcqwc
43 ZTF20acpcuwx
44 ZTF20aciuakq
45 ZTF20acazpve
46 ZTF20acfaonm
47 ZTF20acoykif
48 ZTF20acpbnzr
49 ZTF20acpgvef
50 ZTF20acordqz
51 ZTF20acoyyly
52 ZTF20acgzjmh
53 ZTF20acphbem
54 ZTF20acpjeiu
55 ZTF20acorjop
56 ZTF20acpiwrj
57 ZTF20acoxudw
58 ZTF20acpkled
59 ZTF20acphjfv
60 ZTF20acoawtj
61 ZTF20acoyvdg
62 ZTF20acovrzx
63

489 ZTF19aadavfl
490 ZTF19aadmapf
491 ZTF19aadofaz
492 ZTF19aaeqhjn
493 ZTF19aaesfyl
494 ZTF19abzrcje
495 ZTF19aaefazu
496 ZTF19aailpsf
497 ZTF19aaipokk
498 ZTF19acbyklm
499 ZTF19aazvqod
500 ZTF19aayrhmd
501 ZTF19abzuaoj
502 ZTF19acaxfkb
503 ZTF19accvbkv
504 ZTF20aarbmuu
505 ZTF19accskys
506 ZTF19acbwjjz
507 ZTF19acbwgqj
508 ZTF20abvtvuq
509 ZTF19aamoyzf
510 ZTF19acojuex
511 ZTF19acneyja
512 ZTF20aatvpnx
513 ZTF20acoqrpm
514 ZTF19acrwiye
515 ZTF19aaneseo
516 ZTF19acschjc
517 ZTF20aavudxc
518 ZTF20aavnztx
519 ZTF19acbplek
520 ZTF17aaaocnx
521 ZTF18acurkzx
522 ZTF19abqirpa
523 ZTF18acyrmdh
524 ZTF19aarjspm
525 ZTF19aaqwwde
526 ZTF19aarisna
527 ZTF20abzpxjv
528 ZTF20abjxefy
529 ZTF20aceowmd
530 ZTF20acfnqsh
531 ZTF19abucixu
532 ZTF19aadaqmn
533 ZTF20acgkiwo
534 ZTF20abkdxzy
535 ZTF20aalkbuu
536 ZTF20aajvnji
537 ZTF20aamdzkk
538 ZTF20acgyggs
539 ZTF20abmgzuf
540 ZTF19acvfryu
541 ZTF19abvitks
542 ZTF18adbmhzp
543 ZTF18adbmbgf
544 ZTF19aaaulkx
545 ZTF20aatuqfy
546 ZTF19aawvisn
547 ZTF18adbij

971 ZTF20abqbkzm
972 ZTF19abglrxo
973 ZTF19aauvblr
974 ZTF19aavgyjd
975 ZTF19aavrbra
976 ZTF18adbcjkb
977 ZTF18adccxqw
978 ZTF18adbihrq
979 ZTF18adbiftk
980 ZTF20aammzjf
981 ZTF20abqkeje
982 ZTF20ablmxtl
983 ZTF19aawfxwj
984 ZTF20aatuvvc
985 ZTF19aavriia
986 ZTF20aaunwum
987 ZTF20aaulari
988 ZTF19abyndix
989 ZTF20abbzolz
990 ZTF19aavlnqp
991 ZTF19aaxzglq
992 ZTF19aaflkzn
993 ZTF19aaeprwm
994 ZTF19acsmzfr
995 ZTF19abzvkbz
996 ZTF19acqoxzt
997 ZTF19aafnfis
998 ZTF18aaenucd
999 ZTF18adbhsjc
1000 ZTF19acahpfn
1001 ZTF19acxozlr
1002 ZTF19aazprtn
1003 ZTF19aaznxcw
1004 ZTF19acbmyqe
1005 ZTF19acsxzya
1006 ZTF19abaevmv
1007 ZTF19acdtkbv
1008 ZTF19acajmlj
1009 ZTF19aamvfgy
1010 ZTF20aauwyta
1011 ZTF20aauwysn
1012 ZTF20aauwgnw
1013 ZTF18acqeuat
1014 ZTF20aauolxq
1015 ZTF19aamqory
1016 ZTF19aamgpuc
1017 ZTF19aanhdwf
1018 ZTF19aamtjzm
1019 ZTF19aakjfzu
1020 ZTF20aausfxo
1021 ZTF18aceyycq
1022 ZTF19abgctni
1023 ZTF20aautwvg
1024 ZTF19abgbcgv
1025 ZTF19aancxsc
1026 ZTF19abotlte
1027 ZTF20aaycyka
102

1428 ZTF20actljfs
1429 ZTF20acpumut
1430 ZTF20acthbdd
1431 ZTF20actogyl
1432 ZTF20acqnhci
1433 ZTF20acqwcoi
1434 ZTF18acsjzcl
1435 ZTF20acrufey
1436 ZTF19aacpfqv
1437 ZTF20acqpzbo
1438 ZTF20acrdadi
1439 ZTF20actocyw
1440 ZTF20acrxrfv
1441 ZTF20acrsubb
1442 ZTF20acofydj
1443 ZTF20acpolgu
1444 ZTF20actnuls
1445 ZTF20actdhix
1446 ZTF20acpilio
1447 ZTF20acsesce
1448 ZTF20acrzwvx
1449 ZTF20actiymd
1450 ZTF20actrovz
1451 ZTF20acpbdvm
1452 ZTF20acqgklx
1453 ZTF20actpavu
1454 ZTF20actpatx
1455 ZTF20acsenmb
1456 ZTF20actawpa
1457 ZTF20actskcf
1458 ZTF20actkmks
1459 ZTF20acpjswl
1460 ZTF20acrqeje
1461 ZTF20acpkktq
1462 ZTF20actpbck
1463 ZTF20actoswy
1464 ZTF20acrzuko
1465 ZTF20actvffz
1466 ZTF20acqztpi
1467 ZTF20acuaqlf
1468 ZTF20acuhren
1469 ZTF18acahbqb
1470 ZTF20acucbek
1471 ZTF20actezmo
1472 ZTF20acqnfkh
1473 ZTF20actekrw
1474 ZTF20acufmrl
1475 ZTF20acseuvr
1476 ZTF20actzmzp
1477 ZTF20acrmolb
1478 ZTF20acqikeh
1479 ZTF20acsfdnn
1480 ZTF20actfoxr
1481 ZTF20acqvwzm
1482 ZTF20ackijaj
1483 ZTF20

1884 ZTF21aaaadvr
1885 ZTF21aaagypx
1886 ZTF21aaahtln
1887 ZTF20adadmsg
1888 ZTF20adadsgm
1889 ZTF20adaduxg
1890 ZTF21aaaigfg
1891 ZTF20acrrssy
1892 ZTF21aaaouyd
1893 ZTF21aaaubig
1894 ZTF21aaabtyq
1895 ZTF21aaaovuq
1896 ZTF20acyybwo
1897 ZTF18aarrwmi
1898 ZTF20acyroek
1899 ZTF21aaabvkj
1900 ZTF20acyxzzf
1901 ZTF21aaaollj
1902 ZTF18aczergg
1903 ZTF20acyrodl
1904 ZTF21aaabucr
1905 ZTF20aczgvfe
1906 ZTF21aaaovbc
1907 ZTF21aaapesv
1908 ZTF21aaagzee
1909 ZTF21aaaadmo
1910 ZTF20adagnfh
1911 ZTF20acydasr
1912 ZTF20acasmjl
1913 ZTF19aaqzfuk
1914 ZTF21aabhtvy
1915 ZTF21aaaewsb
1916 ZTF20adadhnz
1917 ZTF20adaftef
1918 ZTF21aaaecrv
1919 ZTF21aaazbpi
1920 ZTF21aaafrnh
1921 ZTF21aaaoslt
1922 ZTF21aaabuoj
1923 ZTF19aadtgaj
1924 ZTF20acyxvbz
1925 ZTF20acwjnux
1926 ZTF20adafawd
1927 ZTF20adadrdh
1928 ZTF20adaenal
1929 ZTF21aaafzwx
1930 ZTF21aaaekui
1931 ZTF21aaaqzta
1932 ZTF20acwztut
1933 ZTF21aaadhjn
1934 ZTF20acxqxyw
1935 ZTF21aaaytyv
1936 ZTF21aaabviz
1937 ZTF21aaaytny
1938 ZTF21aaabzje
1939 ZTF21

2340 ZTF21aakvroo
2341 ZTF21aakjhcb
2342 ZTF21aaldqxj
2343 ZTF21aaiwlyy
2344 ZTF21aajyufk
2345 ZTF21aamaufb
2346 ZTF19acekltt
2347 ZTF21aakijia
2348 ZTF21aagtqkw
2349 ZTF21aajuerd
2350 ZTF21aalydkl
2351 ZTF21aamokak
2352 ZTF21aalodzw
2353 ZTF21aajnefd
2354 ZTF21aaikpnj
2355 ZTF21aalkltz
2356 ZTF21aaluoym
2357 ZTF21aakbgpf
2358 ZTF21aaliivu
2359 ZTF21aalimun
2360 ZTF21aamauao
2361 ZTF21aaloddc
2362 ZTF21aakvhtz
2363 ZTF21aahnqnn
2364 ZTF21aaisube
2365 ZTF21aalnhby
2366 ZTF21aahqahr
2367 ZTF21aagmhzg
2368 ZTF21aalmtom
2369 ZTF21aalnyny
2370 ZTF21aajgdpo
2371 ZTF21aalgqsv
2372 ZTF21aalimya
2373 ZTF21aamgrxm
2374 ZTF21aaiucta
2375 ZTF21aahhars
2376 ZTF21aaekeqv
2377 ZTF21aahaksd
2378 ZTF21aamgcrv
2379 ZTF21aahlbxc
2380 ZTF21aahfjrr
2381 ZTF21aakygxl
2382 ZTF21aakdvst
2383 ZTF21aalepvi
2384 ZTF21aalmsrg
2385 ZTF21aadloxe
2386 ZTF21aahzspd
2387 ZTF21aagskhr
2388 ZTF21aamesud
2389 ZTF21aalofsw
2390 ZTF21aamhtui
2391 ZTF21aamttit
2392 ZTF21aaluqkp
2393 ZTF21aalcrjh
2394 ZTF21aaaagdb
2395 ZTF21

2796 ZTF21aasgcve
2797 ZTF21aaufvpg
2798 ZTF18aatjfoe
2799 ZTF21aasaxfg
2800 ZTF21aatyplr
2801 ZTF21aaunadg
2802 ZTF21aardsgh
2803 ZTF21aatnpnl
2804 ZTF21aartehc
2805 ZTF21aareexg
2806 ZTF21aatdzmt
2807 ZTF21aarzgql
2808 ZTF21aathrnx
2809 ZTF21aaubuml
2810 ZTF21aasbqqw
2811 ZTF21aaugbus
2812 ZTF21aaupkow
2813 ZTF21aatvcdl
2814 ZTF21aasyzkt
2815 ZTF21aavoqpl
2816 ZTF21aatvqcl
2817 ZTF21aatitlf
2818 ZTF21aatpsky
2819 ZTF21aaublej
2820 ZTF21aaufwyh
2821 ZTF21aaufthj
2822 ZTF21aaucuyq
2823 ZTF21aatisro
2824 ZTF21aavidfz
2825 ZTF21aatsafx
2826 ZTF21aaqmoof
2827 ZTF21aatlesr
2828 ZTF21aavacsc
2829 ZTF21aavuprt
2830 ZTF21aavheiv
2831 ZTF21aatwlhj
2832 ZTF21aatjypi
2833 ZTF21aatnxwt
2834 ZTF21aatklhw
2835 ZTF21aauvmck
2836 ZTF21aatkdcw
2837 ZTF21aapdpup
2838 ZTF21aateglh
2839 ZTF21aaskyvp
2840 ZTF21aauamtx
2841 ZTF21aavqwhh
2842 ZTF21aatwkkg
2843 ZTF21aasbkrl
2844 ZTF21aavqphe
2845 ZTF21aawgvpe
2846 ZTF21aavodtn
2847 ZTF21aatiolj
2848 ZTF21aavadbu
2849 ZTF21aaukcog
2850 ZTF21aavhsdw
2851 ZTF21

In [52]:
f = ascii.read("RCF_source.ascii") #ascii file containing the names of sources and their saved dates
sources = f['col1']

#sources = ['ZTF21aagygzv', 'ZTF21aacufip']
    
for source in sources:
    
    flag = 0
    ztfname = source
    
    for i in range (len(get_source_api(ztfname)['comments'])):
        
        comment = get_source_api(ztfname)['comments'][i]['text']
        
        if comment == 'Uploaded to TNS':
            flag = 1
        
    if flag == 0:
        
        print (source)
        
        post_comment(ztfname, 'Uploaded to TNS')

ZTF20acwytxn
ZTF20acqqdkl
ZTF21aacscvb
ZTF21aacpasn
ZTF21aaaaddl
ZTF20acgyjbh
ZTF21aabxztv
ZTF18aajjhkq
ZTF21aadoizf
ZTF21aacvhsc
ZTF21aaateuz
ZTF21aaaosmn
ZTF21aaahyjj
ZTF21aaaewsb
ZTF21aadalct
ZTF21aaczpca
ZTF21aabyqps
ZTF21aabcyvd
ZTF21aaabzje
ZTF21aaabrpu
ZTF20acytfkf
ZTF20aasjzhg
ZTF21aaazieb
ZTF21aaasxel
ZTF21aaaoviv
ZTF21aaabwem
ZTF20acxpvhu
ZTF21aacxung
ZTF21aabfson
ZTF21aaabwfl
ZTF21aabhxxj
ZTF21aaafgmv
ZTF21aaadiap
ZTF21aaabwzx
ZTF21aaaalaf
ZTF20adaghdo
ZTF21aaaldio
ZTF20acymtcs
ZTF21aaaotku
ZTF21aaahuox
ZTF21aaabxln
ZTF21aaabvkj
ZTF21aaabubo
ZTF21aaaawur
ZTF20adaftef
ZTF20acwytht
ZTF21aaagzee
ZTF21aaabaey
ZTF21aaaadmo
ZTF20acztnts
ZTF20acvezdt
ZTF21aaabvit
ZTF20adadlqm
ZTF20aczacag
ZTF20acyrodl
ZTF21aaabvjk
ZTF20adadsgm
ZTF20adadlqv
ZTF20aczgoml
ZTF20acywefl
ZTF20acxzkkf
ZTF20acxtewc
ZTF20acxsxac
ZTF20acxqnhn
ZTF20acwytmh
ZTF20acwphqb
ZTF20acwgnxk
ZTF20acwfftd
ZTF20acweyro
ZTF20acvjlev
ZTF20acveadu
ZTF20acuspji
ZTF20acuoxrx
ZTF20acukxat
ZTF20acueziy
ZTF20acudkao
ZTF20actquzl

In [13]:
def get_obsrun(run_id):
    
    url = BASEURL+'api/observing_run/'+run_id
    response = api('GET',url)
    
    return response

In [14]:
def get_offset(obj_id):
    
    url = BASEURL+'api/sources/'+obj_id+'/offsets'
    
    #data = {  "facility": 'P200'}

    response = api('GET',url)
    #response = api('GET',url,  data=data)
    
    return response

In [15]:
def get_png(obj_id):
    
    url = BASEURL+'api/sources/'+obj_id+'/finder'
    
    #data = {  "facility": 'P200'}

    response = api('GET',url)
    #response = api('GET',url,  data=data)
    
    return response

In [63]:
sources = ['ZTF21aaepqnb', 'ZTF20acpevlh', 'ZTF20abdzjpn', 'ZTF19acbwjjz', 'ZTF19aawmhbu', 'ZTF19aafnfis', 'ZTF19aaednlp']
           
for source in sources:
    
    a = get_IAUname(source)
    
    print (source, a)

ZTF21aaepqnb AT 2021akk
ZTF20acpevlh AT 2020zcj
ZTF20abdzjpn AT 2020lry
ZTF19acbwjjz AT 2019rrd
ZTF19aawmhbu AT 2019hth
ZTF19aafnfis AT 2019aafa
ZTF19aaednlp AT 2019aae


In [86]:
f = ascii.read("RCF_sources_copy.ascii") #ascii file containing the names of sources and their saved dates
sources = f['col1']

#sources = ['ZTF21aagqdvr']

for source in sources:
    
    a = get_all_spectra_len(source)
    
    if a == 0:
        continue
        
    else:
        print (source, a)

ZTF21aaipypa 2
ZTF21aagxmfv 1
ZTF20acqgklx 2
ZTF21aagsihi 1
ZTF21aagpymw 2
ZTF21aagoeny 2
ZTF21aabyifm 1
ZTF20acytgxf 5
ZTF21aagtpro 2
ZTF21aagqcfs 1
ZTF21aagkhwf 1
ZTF21aagbpvf 2
ZTF21aabpnlm 5
ZTF21aaeoqxf 7
ZTF20acpjbgc 2
ZTF21aacvhsc 1
ZTF21aadkhte 8
ZTF21aabjfbc 4
ZTF21aaarmti 1
ZTF21aaarlbp 1
ZTF20acyxrez 3
ZTF21aaabviz 1
ZTF21aaabuoj 1
ZTF21aabpszb 2
ZTF20acyyavb 2
ZTF20acuhgar 2
ZTF20acvebcu 1
ZTF20acrzwvx 2
ZTF21aaaubig 3
ZTF21aaabtyq 1
ZTF20acwpibi 2
ZTF21aaampui 6
ZTF20adadrhw 8
ZTF20acyqzeu 2
ZTF20acxdawc 1
ZTF20acwqqjs 1
ZTF20acwobku 9
ZTF20acuosvy 1
ZTF20acuhren 1
ZTF20actpqgc 6
ZTF20acynllu 2
ZTF20acorjop 1
ZTF20acxvvnf 1
ZTF20acrinvz 3
ZTF20acpjfln 3
ZTF20acynjjo 3
ZTF20acwujgz 2
ZTF20acxzqyp 2
ZTF20acxncha 3
ZTF20actdeqw 4
ZTF20acwmpsj 2
ZTF20acwcfvx 1
ZTF20acvmzfv 2
ZTF20acufmte 3
ZTF20acwgxhk 3
ZTF20acumekm 2
ZTF20actekrw 5
ZTF20acpqbfu 2
ZTF20acvziuf 2
ZTF20acvjagm 10
ZTF20actskcf 2
ZTF20acpvbbh 5
ZTF20acuzfpk 1
ZTF20acphdcg 7
ZTF20acumxrh 2
ZTF20actnuls 4
ZTF20acug

In [18]:
get_classification('ZTF21aaanbla')

('No Classification found', 'None')

In [44]:
len(get_source_api('ZTF20abjgqic')['comments'])

7

In [76]:
get_required_spectrum_id('ZTF21aagsihi')

Please choose from the following spectra: 

1 ) spectrum name:  ZTF21aagsihi_20210215.w.txt spectrum date: 2021-02-16
Choose spectrum to upload: 1


1425

In [81]:
a = get_spectrum_api('1425')
print (a['data'])

{'modified': '2021-02-16T09:58:59.603207', 'assignment_id': None, 'wavelengths': [4020.0, 4029.2, 4038.4, 4047.6, 4056.8, 4066.0, 4075.2, 4084.4, 4093.6, 4102.8, 4112.0, 4121.2, 4130.4, 4139.6, 4148.8, 4158.0, 4167.2, 4176.4, 4185.6, 4194.8, 4204.0, 4213.2, 4222.4, 4231.6, 4240.8, 4250.0, 4259.2, 4268.4, 4277.6, 4286.8, 4296.0, 4305.2, 4314.4, 4323.6, 4332.8, 4342.0, 4351.2, 4360.4, 4369.6, 4378.8, 4388.0, 4397.2, 4406.4, 4415.6, 4424.8, 4434.0, 4443.2, 4452.4, 4461.6, 4470.8, 4480.0, 4489.2, 4498.4, 4507.6, 4516.8, 4526.0, 4535.2, 4544.4, 4553.6, 4562.8, 4572.0, 4581.2, 4590.4, 4599.6, 4608.8, 4618.0, 4627.2, 4636.4, 4645.6, 4654.8, 4664.0, 4673.2, 4682.4, 4691.6, 4700.8, 4710.0, 4719.2, 4728.4, 4737.6, 4746.8, 4756.0, 4765.2, 4774.4, 4783.6, 4792.8, 4802.0, 4811.2, 4820.4, 4829.6, 4838.8, 4848.0, 4857.2, 4866.4, 4875.6, 4884.8, 4894.0, 4903.2, 4912.4, 4921.6, 4930.8, 4940.0, 4949.2, 4958.4, 4967.6, 4976.8, 4986.0, 4995.2, 5004.4, 5013.6, 5022.8, 5032.0, 5041.2, 5050.4, 5059.6, 5068.8